In [1]:
import numpy as np
import tensorflow as tf

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [15]:
class Data_Reader():
    def __init__(self,dataset,batch_size=None):
        npz = np.load('Audiobooks_data_{0}tf.npz'.format(dataset))
        self.inputs = npz['inputs'].astype(np.float32)
        self.targets = npz['inputs'].astype(np.int64)
        if batch_size is None:
            self.batch_size = self.inputs.shape[0]
        else:
            self.batch_size = batch_size
        self.curr_batch = 0
        self.batch_count = self.inputs.shape[0] // self.batch_size
    def __next__(self):
        if self.curr_batch >= self.batch_count:
            self.curr_batch = 0 
            raise StopIteration
        batch_slice = slice(self.curr_batch * self.batch_size,(self.curr_batch+1) * self.batch_size)
        inputs_batch = self.inputs[batch_slice]
        targets_batch = self.targets[batch_slice]
        self.curr_batch +=1
        
        classes_num = 2
        targets_one_hot = np.zeros((targets_batch.shape[0], classes_num))
        targets_one_hot[range(targets_batch.shape[0]), targets_batch] = 1

        return inputs_batch , targets_one_hot
    def __iter__(self):
        return self
        
    

In [ ]:
input_size = 10
output_size = 2
hidden_layer_size = 50

tf.compat.v1.reset_default_graph()


inputs = tf.placeholder(tf.float32,[None,input_size])
targets = tf.placeholder(tf.float32,[None,output_size])


w1 = tf.get_variable('weight1',[input_size,hidden_layer_size])
b1 = tf.get_variable('b1',[hidden_layer_size])

output1 = tf.nn.relu(tf.matmul(inputs,w1)+b1)


w2 = tf.get_variable('weight2',[hidden_layer_size,hidden_layer_size])
b2 = tf.get_variable('b2',[hidden_layer_size])

output2 = tf.nn.relu(tf.matmul(output1,w2) + b2)

w3 = tf.get_variable('weight3',[hidden_layer_size,output_size])
b3 = tf.get_variable('b3',[output_size])

outputs = tf.matmul(output2,w3)+b3

loss = tf.nn.softmax_cross_entropy_with_logits(logits=outputs,labels=targets)

mean_loss = tf.reduce_mean(loss)

optimize = tf.train.AdamOptimizer(learning_rate=.001).minimize(mean_loss)


out_equal_target = tf.equal(tf.argmax(outputs,1),tf.argmax(targets,1))
accuracy = tf.reduce_mean(tf.cast(out_equal_target,tf.float32))

sess = tf.InteractiveSession()
initializer = tf.global_variables_initializer()
sess.run(initializer)
batch_size = 100

max_epochs = 15
prev_val_loss = 9999999.


train_data = Data_Reader('train', batch_size)
validation_data = Data_Reader ('validation')

In [ ]:
for epoch_counter in range(max_epochs):
    curr_epoch_loss = 0.
    for input_batch, target_batch in train_data:
        _,batch_loss = sess.run([optimize,mean_loss],
                                feed_dict = {inputs:input_batch,targets:target_batch})
        curr_epoch_loss = batch_loss
        curr_epoch_loss /= train_data.batch_count
    validation_loss = 0.
    validation_accuracy = 0.
    
    for input_batch, target_batch in validation_data:
        validation_loss,validation_accuracy = sess.run([mean_loss,accuracy],
                                feed_dict = {inputs:input_batch,targets:target_batch})
    print('Epochs'+str(epoch_counter+1)+'. Training loss: ' + '{0:3f}'.format(curr_epoch_loss)+'validation loss: ' +'{0:.3f}'.format(validation_loss)+'validation accuracy'+'{0.2f}'.format(validation_accuracy*100)+'%')
    
    
    if validation_loss > prev_val_loss:
        break
    
    prev_val_loss = validation_loss
    
    print("End of training")
        
        
        